In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
from CreateWebLogo import createWebLogo
tk.Tk().withdraw()
bioinformaticsDir = filedialog.askdirectory(title = "Choose Bioinformatics Directory")
deaminationDeterminationDataDir = os.path.join(bioinformaticsDir, "deamination_determination", "data")
generalReadSequencesDirectories = dict()
cellTypes = ("NHF1", "Arabidopsis", "yeast", "XP-C", "CS-B", "E_coli_WT")
timepointsByCellType = {"NHF1":["1h","8h","24h","48h"], "Arabidopsis":["ZT2"], "yeast":["20min"], "XP-C":["1h"], "CS-B":["1h"], "E_coli_WT":["5min"]}
for cellType in cellTypes:
    generalReadSequencesDirectories[cellType] = os.path.join(deaminationDeterminationDataDir, cellType, "general_read_sequences")

In [ ]:
CG_ContentByCellType = dict()
for cellType in cellTypes:
    for timepoint in timepointsByCellType[cellType]:
        readsFilePath = os.path.join(generalReadSequencesDirectories[cellType],
                                     f"{cellType}_CPD_{timepoint}_all_reps_aligned_reads_10bp_expanded_TGG_filtered.bed")
        nucleotideCounts = dict()
        with open(readsFilePath, 'r') as readsFile:
            for line in readsFile:
                readSequence = line.split()[6]
                for nucleotide in readSequence:
                    nucleotideCounts[nucleotide] = nucleotideCounts.setdefault(nucleotide, 0) + 1
        
        CG_ContentByCellType[cellType] = ((nucleotideCounts["C"] + nucleotideCounts["G"]) /
                                        (nucleotideCounts["A"] + nucleotideCounts["C"] +
                                        nucleotideCounts["G"] + nucleotideCounts["T"]) * 100)
        print(f"Nucleotide counts for {os.path.basename(readsFilePath)}:")
        for key,value in nucleotideCounts.items():  print(f"\t{key}: {value}")
        print(f"CG content: {CG_ContentByCellType[cellType]}\n")

In [ ]:
#NOTE: Web logo creation with the current software suuuuuuuuucks. Maybe find an alternative if we want to do high-throughput analysis like this?
for cellType in cellTypes:
    for strandPolarity in ("three_prime", "five_prime"):
        if strandPolarity == "five_prime" and (cellType == "Arabidopsis" or cellType == "yeast"): continue
        print(f"Creating {strandPolarity.replace('_',' ')} sequence logo for {cellType}")
        webLogoInputFilePath = os.path.join(
            generalReadSequencesDirectories[cellType],
            f"{cellType}_CPD_{timepointsByCellType[cellType][0]}_all_reps_{strandPolarity}"
             "_sequence_logo_input_TGG_filtered_nonstandard_nuc_filtered.txt"
        )
        createWebLogo([webLogoInputFilePath], round(CG_ContentByCellType[cellType]))
    